# Lab 3: Personal Website Chatbot with Quality Control

## Lab Objectives

In this lab, you'll build an intelligent chatbot that can answer questions about a personal website or portfolio. The chatbot will include quality control features to ensure helpful responses.

**What you'll learn:**
- Building domain-specific chatbots
- Quality control and evaluation systems
- PDF processing and content extraction
- Advanced prompting techniques
- Multi-model comparison approaches

**What you'll build:**
- A chatbot that knows about your personal/professional content
- Quality evaluation system for chatbot responses  
- A simple web interface using Gradio
- Comparison between different AI models

# Section 1: Environment Setup and Content Loading

In [ ]:
import os
import openai
from openai import OpenAI
from pydantic import BaseModel
from typing import Optional
import pypdf
import gradio as gr

print("Starting Lab 3: Personal Website Chatbot")
print("Libraries imported successfully")

In [ ]:
# Check available API keys
required_vars = ['OPENAI_API_KEY', 'GOOGLE_API_KEY', 'DASHSCOPE_API_KEY']
missing_vars = []

print("API Key Status Check:")
for var in required_vars:
    if os.getenv(var):
        print(f"[OK] {var}: Available")
    else:
        print(f"[--] {var}: Not set")
        missing_vars.append(var)

if missing_vars:
    print(f"\nMissing API keys: {', '.join(missing_vars)}")
    print("You can still run this lab with just one API key!")
else:
    print("\nAll API keys available - you can use any path!")

✅ Environment variables loaded
📋 Available APIs will depend on your chosen path


In [ ]:
# Load your personal/professional content from PDF
# Replace 'your_resume.pdf' with your actual PDF file path

pdf_path = 'your_resume.pdf'  # Change this to your PDF file

try:
    with open(pdf_path, 'rb') as f:
        reader = pypdf.PdfReader(f)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    
    print(f"[SUCCESS] PDF loaded: {len(text)} characters extracted")
    print(f"First 200 characters: {text[:200]}...")

except FileNotFoundError:
    print("[INFO] PDF file not found - using sample content instead")
    
    # Sample professional content if no PDF is available
    text = """
    John Smith - Software Developer
    
    Professional Experience:
    - 5 years of experience in Python development
    - Specialized in AI/ML applications and web development
    - Led teams of 3-5 developers on multiple projects
    - Experience with OpenAI API, machine learning frameworks
    
    Skills:
    - Programming: Python, JavaScript, SQL
    - AI/ML: OpenAI API, scikit-learn, TensorFlow
    - Web: React, Flask, Django
    - Tools: Git, Docker, AWS
    
    Education:
    - Bachelor's in Computer Science
    - Certified in Cloud Computing
    
    Projects:
    - Built an AI-powered chatbot for customer service
    - Developed a machine learning model for predictive analytics  
    - Created a full-stack web application with 10,000+ users
    """
    
    print("[INFO] Using sample content for demonstration")

print(f"Total content length: {len(text)} characters")

✅ LinkedIn profile loaded (8256 characters)
📄 Replace me/linkedin.pdf with your own profile!


In [ ]:
# Add LinkedIn profile information (optional)
linkedin = """
LinkedIn: linkedin.com/in/johnsmith-dev

Recent posts and activities:
- Shared insights about AI in software development
- Posted about recent project using OpenAI API
- Engaged with posts about Python best practices
- Connected with other developers and AI enthusiasts

Recommendations received:
"John is an exceptional developer with strong AI/ML skills. His work on our chatbot project was outstanding." - Former Manager

"Collaborative team player who consistently delivers high-quality code." - Colleague
"""

# Combine all content
summary = text + "\n\n" + linkedin

print("[INFO] Content preparation complete")
print(f"Total knowledge base: {len(summary)} characters")
print("\nContent includes:")
print("- Professional experience and skills")  
print("- Education and certifications")
print("- Project portfolio")
print("- LinkedIn activity and recommendations")

   
Contact
ed.donner@gmail.com
www.linkedin.com/in/eddonner
(LinkedIn)
edwarddonner.com (Personal)
Top Skills
CTO
Large Language Models (LLM)
PyTorch
Patents
Apparatus for determining role
fitness while eliminating unwanted
bias
Ed Donner
Co-Founder & CTO at Nebula.io, repeat Co-Founder of AI startups,
speaker & advisor on Gen AI and LLM Engineering
New York, New York, United States
Summary
I’m a technology leader and entrepreneur. I'm applying AI to a field
where it can make a massive impact: helping people discover their
potential and pursue their reason for being. But at my core, I’m a
software engineer and a scientist. I learned how to code aged 8 and
still spend weekends experimenting with Large Language Models
and writing code (rather badly). If you’d like to join us to show me
how it’s done.. message me!
As a work-hobby, I absolutely love giving talks about Gen AI and
LLMs. I'm the author of a best-selling, top-rated Udemy course
on LLM Engineering, and I speak at O'Reilly Live

In [ ]:
# Setup AI clients based on available API keys
clients = {}

# OpenAI setup
if os.getenv('OPENAI_API_KEY'):
    openai = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    clients['OpenAI'] = openai
    print("[SETUP] OpenAI client configured")

# Google Gemini setup (using OpenAI-compatible interface)
if os.getenv('GOOGLE_API_KEY'):
    gemini = OpenAI(
        api_key=os.getenv('GOOGLE_API_KEY'),
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    clients['Gemini'] = gemini
    print("[SETUP] Gemini client configured")

# Qwen setup
if os.getenv('DASHSCOPE_API_KEY'):
    qwen_chat = OpenAI(
        api_key=os.getenv('DASHSCOPE_API_KEY'),
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
    )
    clients['Qwen'] = qwen_chat
    print("[SETUP] Qwen client configured")

print(f"\nTotal clients available: {len(clients)}")
if not clients:
    print("[WARNING] No API keys found - please configure at least one API key")

✅ Personal summary loaded (387 characters)
📝 Update me/summary.txt with your own background!


In [ ]:
# Test the chatbot with different clients
system_prompt = f"""You are a helpful assistant that answers questions about this person's professional background and experience. Here is their information:

{summary}

Guidelines:
- Answer questions based only on the provided information
- Be friendly and professional
- If asked about something not in the provided info, say "I don't have that specific information"
- Keep responses concise but informative
"""

def test_chatbot(question, client_name):
    """Test the chatbot with a specific client"""
    if client_name not in clients:
        return f"Client {client_name} not available"
    
    client = clients[client_name]
    
    try:
        # Select appropriate model based on client
        if client_name == 'OpenAI':
            model = "gpt-4o-mini"
        elif client_name == 'Gemini':
            model = "gemini-1.5-flash"
        elif client_name == 'Qwen':
            model = "qwen2.5-72b-instruct"
        
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question}
            ],
            max_tokens=200
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"Error with {client_name}: {str(e)}"

# Test with first available client
test_question = "What programming languages does this person know?"

for client_name in clients.keys():
    print(f"\n[TESTING] {client_name} Response:")
    print("-" * 40)
    test_reply = test_chatbot(test_question, client_name)
    print(test_reply)
    break  # Test with just the first available client

✅ Chatbot will represent: Ed Donner
⚠️  Remember to change this to YOUR name!


In [ ]:
# Quality control evaluation system
class Evaluation(BaseModel):
    accuracy: int  # 1-10 scale
    helpfulness: int  # 1-10 scale  
    completeness: int  # 1-10 scale
    overall_score: int  # 1-10 scale
    feedback: str

evaluator_system_prompt = """You are an expert evaluator assessing chatbot responses about professional profiles. 

Rate each response on:
- Accuracy (1-10): How factually correct is the response?
- Helpfulness (1-10): How useful is this response to the user?
- Completeness (1-10): How complete is the answer?
- Overall Score (1-10): Your overall assessment

Provide constructive feedback explaining your scores."""

def evaluate_response(question, response, evaluator_client):
    """Evaluate a chatbot response for quality"""
    
    evaluation_prompt = f"""
Question: {question}
Response to evaluate: {response}

Please evaluate this chatbot response using the criteria above. Return your evaluation in the specified JSON format.
"""
    
    try:
        # Use OpenAI for evaluation if available, otherwise Qwen
        if evaluator_client == 'OpenAI' and 'OpenAI' in clients:
            eval_response = clients['OpenAI'].chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": evaluator_system_prompt},
                    {"role": "user", "content": evaluation_prompt}
                ],
                max_tokens=300
            )
        elif evaluator_client == 'Qwen' and 'Qwen' in clients:
            eval_response = clients['Qwen'].chat.completions.create(
                model="qwen2.5-72b-instruct", 
                messages=[
                    {"role": "system", "content": evaluator_system_prompt},
                    {"role": "user", "content": evaluation_prompt}
                ],
                max_tokens=300
            )
        else:
            return "No evaluator client available"
            
        eval_text = eval_response.choices[0].message.content
        
        # Try to extract structured evaluation
        if "accuracy" in eval_text.lower():
            return eval_text
        else:
            return f"Evaluation completed: {eval_text}"
            
    except Exception as e:
        return f"Evaluation error: {str(e)}"

# Test the evaluation system
print("[TESTING] Quality Control System")
print("=" * 40)

if test_reply:
    # Choose evaluator (prefer OpenAI, fallback to Qwen)
    evaluator = 'OpenAI' if 'OpenAI' in clients else 'Qwen' if 'Qwen' in clients else None
    
    if evaluator:
        print(f"Using {evaluator} as evaluator")
        evaluation = evaluate_response(test_question, test_reply, evaluator)
        print("\nEVALUATION RESULTS:")
        print(evaluation)
    else:
        print("No evaluator available")
else:
    print("No response to evaluate")

✅ System prompt created
📏 Prompt length: 9305 characters
🎭 Your chatbot now has personality and context!


# Section 2: Option 1 - Multi-Model Approach (OpenAI + Gemini)

This section demonstrates using multiple AI models with quality control. It compares responses from different models and provides evaluation feedback.

In [ ]:
def enhanced_chatbot(question):
    """Enhanced chatbot that uses multiple models and evaluation"""
    
    print(f"QUESTION: {question}")
    print("=" * 50)
    
    responses = {}
    evaluations = {}
    
    # Get responses from available models
    for client_name in ['OpenAI', 'Gemini']:
        if client_name in clients:
            print(f"\n[{client_name}] Generating response...")
            response = test_chatbot(question, client_name)
            responses[client_name] = response
            
            # Evaluate the response
            evaluator = 'OpenAI' if 'OpenAI' in clients else 'Qwen'
            if evaluator in clients:
                evaluation = evaluate_response(question, response, evaluator)
                evaluations[client_name] = evaluation
    
    # Display results
    for client_name, response in responses.items():
        print(f"\n[RESPONSE - {client_name}]")
        print("-" * 30)
        print(response)
        
        if client_name in evaluations:
            print(f"\n[EVALUATION - {client_name}]")
            print(evaluations[client_name])
            print("-" * 30)
    
    return responses, evaluations

# Test the enhanced system
if len(clients) >= 1:
    print("Testing enhanced multi-model chatbot...")
    
    test_questions = [
        "What is this person's main area of expertise?",
        "What kind of projects has this person worked on?", 
        "What are their technical skills?"
    ]
    
    for question in test_questions[:1]:  # Test with first question
        results = enhanced_chatbot(question)
        print("\n" + "="*60)
        
else:
    print("Need at least one API client to run enhanced chatbot")

# 📚 Section 3: Test Basic Chat (OpenAI Path)

Let's test the basic chatbot before adding quality control.

In [ ]:
# Test the basic OpenAI chatbot (without quality control)
print("🚀 Launching basic OpenAI chatbot for testing...")
print("🧪 This is just a test - we'll add quality control next")

gr.ChatInterface(chat_openai, type="messages").launch()

# Section 3: Option 2 - Complete Qwen Solution

In [ ]:
# Complete Qwen-based solution
if os.getenv('DASHSCOPE_API_KEY'):
    print("[QWEN SETUP] Configuring complete Qwen solution...")
    
    # Main chatbot client  
    qwen_chat = OpenAI(
        api_key=os.getenv('DASHSCOPE_API_KEY'),
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
    )
    
    # Separate evaluator client (using same API but different system prompt)
    qwen_evaluator = OpenAI(
        api_key=os.getenv('DASHSCOPE_API_KEY'),
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
    )
    
    print("[QWEN SETUP] Chat client configured")
    print("[QWEN SETUP] Evaluator client configured")
    print("[QWEN SETUP] Ready for complete Qwen workflow")
    
else:
    print("[ERROR] DASHSCOPE_API_KEY not found!")
    print("Please configure your Qwen API key to use this section.")

✅ Qwen clients ready!
💬 Chat model: qwen-plus
🔍 Evaluator model: qwen-max


In [ ]:
def qwen_chatbot(question):
    """Pure Qwen chatbot implementation"""
    
    if not os.getenv('DASHSCOPE_API_KEY'):
        return "Qwen API key not configured"
    
    try:
        response = qwen_chat.chat.completions.create(
            model="qwen2.5-72b-instruct",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question}
            ],
            max_tokens=250
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"Qwen error: {str(e)}"

def qwen_evaluate(question, response):
    """Qwen-based evaluation"""
    
    if not os.getenv('DASHSCOPE_API_KEY'):
        return "Qwen evaluator not available"
    
    evaluation_prompt = f"""
Question: {question}
Response to evaluate: {response}

Please evaluate this response on a scale of 1-10 for:
- Accuracy: How factually correct is it?
- Helpfulness: How useful is it to the user? 
- Completeness: How complete is the answer?

Provide scores and brief feedback.
"""
    
    try:
        eval_response = qwen_evaluator.chat.completions.create(
            model="qwen2.5-72b-instruct",
            messages=[
                {"role": "system", "content": evaluator_system_prompt},
                {"role": "user", "content": evaluation_prompt}
            ],
            max_tokens=200
        )
        
        return eval_response.choices[0].message.content
        
    except Exception as e:
        return f"Evaluation error: {str(e)}"

# Test Qwen system
if os.getenv('DASHSCOPE_API_KEY'):
    print("Testing complete Qwen solution...")
    print("=" * 40)
    
    qwen_question = "What are this person's key technical skills?"
    
    print(f"Question: {qwen_question}")
    print("\n[QWEN RESPONSE]")
    qwen_reply = qwen_chatbot(qwen_question)
    print(qwen_reply)
    
    print("\n[QWEN EVALUATION]") 
    qwen_eval = qwen_evaluate(qwen_question, qwen_reply)
    print(qwen_eval)
    
else:
    print("Qwen API key required for this section")

=== Checking Qwen Version Dependencies ===
✅ name: Available
✅ summary: Available
✅ linkedin: Available
✅ system_prompt: Available
✅ DASHSCOPE_API_KEY: Available (sk-0958b...)

🎉 ALL DEPENDENCIES READY! Qwen version can proceed.


In [ ]:
def qwen_with_context_awareness(question):
    """Enhanced Qwen chatbot with better context awareness"""
    
    enhanced_prompt = f"""You are an expert assistant answering questions about a professional's background. Here's their information:

{summary}

Key instructions:
- Base your answers strictly on the provided information
- If information is missing, clearly state what you don't know
- Provide specific examples when available
- Be conversational but professional
- Focus on the most relevant details for each question

Question: {question}
"""

    if not os.getenv('DASHSCOPE_API_KEY'):
        return "Qwen API not configured"
    
    try:
        response = qwen_chat.chat.completions.create(
            model="qwen2.5-72b-instruct",
            messages=[{"role": "user", "content": enhanced_prompt}],
            max_tokens=300
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"Error: {str(e)}"

def qwen_batch_qa(questions):
    """Process multiple questions efficiently"""
    
    print("[BATCH PROCESSING] Processing multiple questions...")
    results = {}
    
    for i, question in enumerate(questions, 1):
        print(f"\n[Question {i}/{len(questions)}] {question}")
        print("-" * 30)
        
        response = qwen_with_context_awareness(question)
        results[question] = response
        
        print(response)
    
    return results

# Test enhanced features
if os.getenv('DASHSCOPE_API_KEY'):
    test_questions = [
        "What is this person's educational background?",
        "What projects have they worked on?",
        "What makes them a good candidate for a software role?"
    ]
    
    print("Testing enhanced Qwen features...")
    batch_results = qwen_batch_qa(test_questions)
    
else:
    print("Qwen API key needed for enhanced features")

✅ LinkedIn PDF loaded successfully
✅ Summary loaded successfully
✅ Name set to: Ed Donner
✅ System prompt created

🎉 All Qwen dependencies now loaded! You can proceed with the Qwen cells below.


In [ ]:
def qwen_quality_system(question, quality_threshold=7):
    """Complete quality control system using only Qwen"""
    
    print(f"[QUALITY SYSTEM] Processing: {question}")
    print(f"Quality threshold: {quality_threshold}/10")
    print("=" * 45)
    
    # Get response
    print("[STEP 1] Generating response...")
    response = qwen_with_context_awareness(question)
    
    # Evaluate response
    print("[STEP 2] Evaluating quality...")
    evaluation = qwen_evaluate(question, response)
    
    # Simple quality check (in practice, you'd parse the evaluation)
    quality_indicators = [
        len(response) > 100,  # Substantial response
        "don't know" not in response.lower() or "information" in response.lower(),  # Informative
        any(keyword in response.lower() for keyword in ['experience', 'skills', 'project', 'work'])  # Relevant
    ]
    
    quality_score = sum(quality_indicators) * 3  # Simple scoring
    
    print(f"[STEP 3] Quality score: {quality_score}/10")
    
    if quality_score >= quality_threshold:
        print("[SUCCESS] Quality threshold met!")
        status = "APPROVED"
    else:
        print("[WARNING] Below quality threshold")
        status = "NEEDS_IMPROVEMENT"
    
    return {
        'question': question,
        'response': response,
        'evaluation': evaluation,
        'quality_score': quality_score,
        'status': status
    }

# Test complete quality system
if os.getenv('DASHSCOPE_API_KEY'):
    print("Testing complete Qwen quality system...")
    
    quality_test = qwen_quality_system(
        "What programming experience does this person have?",
        quality_threshold=6
    )
    
    print("\n[FINAL RESULTS]")
    print("=" * 40)
    print(f"Status: {quality_test['status']}")
    print(f"Quality Score: {quality_test['quality_score']}")
    print(f"\nResponse:\n{quality_test['response']}")
    
else:
    print("Complete Qwen system requires DASHSCOPE_API_KEY")

=== Testing Qwen API Connection ===
✅ Qwen API connection successful!
Response: Hello, Qwen API works!
✅ Qwen API connection successful!
Response: Hello, Qwen API works!


In [ ]:
def compare_all_approaches(question):
    """Compare responses from all available models"""
    
    print(f"[COMPARISON] Question: {question}")
    print("=" * 50)
    
    results = {}
    
    # Test each available model
    available_models = {
        'OpenAI': lambda q: test_chatbot(q, 'OpenAI') if 'OpenAI' in clients else None,
        'Gemini': lambda q: test_chatbot(q, 'Gemini') if 'Gemini' in clients else None,
        'Qwen': lambda q: qwen_chatbot(q) if os.getenv('DASHSCOPE_API_KEY') else None
    }
    
    for model_name, model_func in available_models.items():
        response = model_func(question)
        if response:
            results[model_name] = response
            print(f"\n[{model_name}] Response:")
            print("-" * 25)
            print(response)
    
    print(f"\n[SUMMARY] Tested {len(results)} models")
    return results

# Run comparison if any models are available
if clients or os.getenv('DASHSCOPE_API_KEY'):
    comparison_question = "What is this person's main expertise?"
    comparison_results = compare_all_approaches(comparison_question)
else:
    print("No models available for comparison")

In [ ]:
def simulate_conversation():
    """Simulate a conversation flow"""
    
    conversation = [
        "Hi! Can you tell me about this person's background?",
        "What kind of projects have they worked on?", 
        "What are their main technical skills?",
        "What makes them qualified for AI/ML roles?"
    ]
    
    print("[CONVERSATION SIMULATION]")
    print("=" * 40)
    
    # Choose best available model
    if 'OpenAI' in clients:
        model_func = lambda q: test_chatbot(q, 'OpenAI')
        model_name = "OpenAI"
    elif os.getenv('DASHSCOPE_API_KEY'):
        model_func = qwen_chatbot
        model_name = "Qwen"
    else:
        print("No models available for conversation")
        return
    
    print(f"Using {model_name} for conversation")
    print("-" * 40)
    
    for i, question in enumerate(conversation, 1):
        print(f"\n[Turn {i}] User: {question}")
        response = model_func(question)
        print(f"[Turn {i}] Assistant: {response}")
        print("-" * 30)

# Run conversation simulation
simulate_conversation()

In [ ]:
# Section 4: Web Interface and Deployment
    
    print(f"Analyzing {model_name} performance...")
    
    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\n[Scenario {i}] {scenario}")
        print("-" * 35)
        
        response = model_func(scenario)
        
        # Simple analysis metrics
        response_length = len(response)
        has_specific_info = any(word in response.lower() 
                              for word in ['python', 'javascript', 'experience', 'project'])
        
        print(f"Response: {response}")
        print(f"Length: {response_length} characters")
        print(f"Contains specific info: {'Yes' if has_specific_info else 'No'}")
    
    print(f"\n[ANALYSIS COMPLETE] Tested {len(test_scenarios)} scenarios")

analyze_performance()

# 🌟 Section 5: Complete Qwen Version (Alternative Path)

**Choose this path if you want a fully self-contained solution with Qwen models!**

## 🎯 What This Path Offers:
- **Qwen-Plus** for main chat functionality (fast, smart responses)
- **Qwen-Max** for evaluation (premium model for quality control)
- **No geographic restrictions** - works globally
- **Self-contained** - only needs DASHSCOPE_API_KEY
- **Same quality control** - automatic evaluation and retry

## 📋 Prerequisites:
- `DASHSCOPE_API_KEY` in your `.env` file
- All Section 1 cells completed (data loading)

## 🚀 Quick Start:
1. Run the dependency check below
2. If needed, run the complete setup
3. Test connectivity  
4. Launch the Qwen chatbot!

**Note:** You can run this section independently if you've completed Section 1.

In [ ]:
def create_gradio_interface():
    """Create a web interface for the chatbot"""
    
    def chatbot_interface(question, model_choice):
        """Handle chatbot requests from web interface"""
        
        if not question.strip():
            return "Please enter a question!"
        
        try:
            if model_choice == "OpenAI" and 'OpenAI' in clients:
                response = test_chatbot(question, 'OpenAI')
            elif model_choice == "Gemini" and 'Gemini' in clients:
                response = test_chatbot(question, 'Gemini')
            elif model_choice == "Qwen" and os.getenv('DASHSCOPE_API_KEY'):
                response = qwen_chatbot(question)
            else:
                response = f"{model_choice} not available. Please configure the API key."
            
            return response
            
        except Exception as e:
            return f"Error: {str(e)}"
    
    # Create model options based on available APIs
    model_options = []
    if 'OpenAI' in clients:
        model_options.append("OpenAI")
    if 'Gemini' in clients:
        model_options.append("Gemini")
    if os.getenv('DASHSCOPE_API_KEY'):
        model_options.append("Qwen")
    
    if not model_options:
        print("[ERROR] No models available for web interface")
        return None
    
    print(f"[GRADIO] Available models: {', '.join(model_options)}")
    
    # Create the interface
    interface = gr.Interface(
        fn=chatbot_interface,
        inputs=[
            gr.Textbox(
                label="Ask a question about the professional profile",
                placeholder="e.g., What are their technical skills?",
                lines=2
            ),
            gr.Dropdown(
                choices=model_options,
                label="Choose AI Model",
                value=model_options[0]
            )
        ],
        outputs=gr.Textbox(label="Response", lines=5),
        title="Personal Website Chatbot",
        description="Ask questions about the professional background and experience.",
        examples=[
            ["What programming languages does this person know?", model_options[0]],
            ["What kind of projects have they worked on?", model_options[0]],
            ["What is their educational background?", model_options[0]]
        ]
    )
    
    return interface

# Create and launch the interface
print("Creating web interface...")
interface = create_gradio_interface()

if interface:
    print("[SUCCESS] Web interface created!")
    print("Run the next cell to launch the interface")
else:
    print("[ERROR] Could not create interface - no models available")

✅ Qwen clients configured!
💬 Chat model: qwen-plus
🔍 Evaluator model: qwen-max
🌍 Global access - no geographic restrictions


In [ ]:
# Launch the web interface
if 'interface' in locals() and interface is not None:
    print("Launching chatbot web interface...")
    print("The interface will open in your browser")
    print("Use Ctrl+C to stop the server when done")
    
    # Launch with share=False for local use only
    # Set share=True if you want to create a public link
    interface.launch(
        share=False,  # Set to True for public sharing
        server_name="127.0.0.1",  # Local access only
        server_port=7860  # Default Gradio port
    )
else:
    print("Interface not available - please run the previous cell first")

✅ Qwen evaluation system ready
🔍 Qwen-Max will judge response quality
🛡️  Robust error handling included


In [ ]:
def create_enhanced_interface():
    """Create an enhanced interface with quality control"""
    
    def enhanced_chatbot_interface(question, model_choice, include_evaluation):
        """Enhanced chatbot with optional evaluation"""
        
        if not question.strip():
            return "Please enter a question!", ""
        
        try:
            # Get response
            if model_choice == "OpenAI" and 'OpenAI' in clients:
                response = test_chatbot(question, 'OpenAI')
            elif model_choice == "Gemini" and 'Gemini' in clients:
                response = test_chatbot(question, 'Gemini') 
            elif model_choice == "Qwen" and os.getenv('DASHSCOPE_API_KEY'):
                response = qwen_chatbot(question)
            else:
                return f"{model_choice} not available", ""
            
            # Get evaluation if requested
            evaluation = ""
            if include_evaluation:
                if 'OpenAI' in clients:
                    evaluation = evaluate_response(question, response, 'OpenAI')
                elif os.getenv('DASHSCOPE_API_KEY'):
                    evaluation = qwen_evaluate(question, response)
                else:
                    evaluation = "Evaluation not available"
            
            return response, evaluation
            
        except Exception as e:
            return f"Error: {str(e)}", ""
    
    # Get available models
    model_options = []
    if 'OpenAI' in clients:
        model_options.append("OpenAI")
    if 'Gemini' in clients:
        model_options.append("Gemini")
    if os.getenv('DASHSCOPE_API_KEY'):
        model_options.append("Qwen")
    
    if not model_options:
        print("[ERROR] No models available")
        return None
    
    print(f"[ENHANCED] Creating interface with models: {', '.join(model_options)}")
    
    # Create enhanced interface
    enhanced_interface = gr.Interface(
        fn=enhanced_chatbot_interface,
        inputs=[
            gr.Textbox(
                label="Your Question",
                placeholder="Ask about technical skills, experience, projects...",
                lines=3
            ),
            gr.Dropdown(
                choices=model_options,
                label="AI Model",
                value=model_options[0]
            ),
            gr.Checkbox(
                label="Include Quality Evaluation",
                value=False
            )
        ],
        outputs=[
            gr.Textbox(label="Chatbot Response", lines=6),
            gr.Textbox(label="Quality Evaluation", lines=4)
        ],
        title="Enhanced Personal Chatbot",
        description="Professional background chatbot with optional quality evaluation",
        examples=[
            ["What technical skills does this person have?", model_options[0], True],
            ["Tell me about their project experience", model_options[0], False],
            ["What makes them qualified for AI roles?", model_options[0], True]
        ]
    )
    
    return enhanced_interface

# Create enhanced interface
print("Creating enhanced interface with quality control...")
enhanced_interface = create_enhanced_interface()

if enhanced_interface:
    print("[SUCCESS] Enhanced interface ready!")
    print("This interface includes quality evaluation features")
else:
    print("[ERROR] Could not create enhanced interface")

✅ Complete Qwen workflow ready!
🤖 Qwen-Plus chat + 🔍 Qwen-Max evaluation + 🔄 Auto-retry


In [ ]:
# Deployment considerations and tips
print("DEPLOYMENT CONSIDERATIONS")
print("=" * 40)

deployment_tips = [
    "Security: Never expose API keys in client-side code",
    "Rate Limiting: Implement request limits to control costs",
    "Error Handling: Always handle API failures gracefully", 
    "Monitoring: Track usage and response quality over time",
    "Content Updates: Plan for updating the knowledge base",
    "User Experience: Provide clear instructions and examples"
]

for i, tip in enumerate(deployment_tips, 1):
    print(f"{i}. {tip}")

print("\nDEPLOYMENT OPTIONS:")
print("-" * 20)
print("• Local: Run on your computer (current setup)")
print("• Cloud: Deploy to Heroku, Vercel, or similar")
print("• Enterprise: Integrate into existing websites")

print("\nNEXT STEPS:")
print("-" * 15)
print("• Test with real questions from your domain")
print("• Expand the knowledge base with more content")
print("• Add authentication if deploying publicly")
print("• Monitor and improve response quality")

🧪 Testing Qwen-Plus + Qwen-Max quality control...
✅ Qwen API connection confirmed
🔍 Evaluation test - Acceptable: False
💬 Feedback: The response is inaccurate and misses an opportunity to highlight relevant information. Ed Donner's LinkedIn profile explicitly mentions a patented invention related to AI and talent, developed during his time at untapt. The Agent should have acknowledged this patent and provided details about it, since it is directly relevant to the question. Additionally, the response comes across as evasive rather than engaging or helpful, which does not align with the professional and informative tone expected when representing Ed Donner.


In [ ]:
# Launch the enhanced interface (optional)
# Uncomment the lines below to launch the enhanced interface

# if 'enhanced_interface' in locals() and enhanced_interface is not None:
#     print("Launching enhanced interface...")
#     enhanced_interface.launch(share=False, server_port=7861)
# else:
#     print("Enhanced interface not available")

print("Enhanced interface ready to launch!")
print("Uncomment the code above to start the enhanced web interface")
print("Note: This will run on port 7861 to avoid conflicts")

🚀 Launching Qwen-powered personal website chatbot...
💬 Main chat: Qwen-Plus (fast, efficient)
🔍 Quality control: Qwen-Max (premium evaluation)
🔄 Auto-retry on failed evaluation
🌍 Global access - no geographic restrictions

Try asking about patents to see the pig latin feature!
* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


✅ Passed Qwen-Max evaluation - returning reply


# Conclusion

Excellent work! You've successfully built a sophisticated personal website chatbot with quality control features. Here's what you accomplished:

**Technical Skills Demonstrated:**
- PDF processing and content extraction
- Multi-model AI integration (OpenAI, Gemini, Qwen)
- Quality evaluation systems
- Web interface development with Gradio
- Error handling and retry logic

**AI Agent Concepts Learned:**
- Domain-specific knowledge integration
- Response quality evaluation
- Multi-model comparison and selection
- Automated quality control systems
- User interface design for AI applications

**Real-World Applications:**
- Personal portfolio chatbots
- Customer service automation
- Knowledge base querying systems
- Educational content assistants
- Professional screening tools

**Key Takeaways:**
- Quality control is essential for production AI systems
- Different models have different strengths for different tasks
- User experience design matters as much as AI performance
- Proper error handling makes systems more robust
- Content preparation significantly impacts chatbot quality

**Next Steps:**
- Expand to handle multiple document types (PDFs, websites, databases)
- Add conversation memory for multi-turn interactions
- Implement more sophisticated evaluation metrics
- Add user feedback collection for continuous improvement
- Explore integration with vector databases for larger knowledge bases

You now have a solid foundation for building production-ready AI chatbot systems!